In [70]:
# This file contains readers for the data used in the project
import pandas as pd

# Data sources
# Electricity: https://porssisahko.net/tilastot
# weather: https://www.ilmatieteenlaitos.fi/havaintojen-lataus
ep_df = pd.read_excel("Electricity_price-010121-220924.xlsx", header = 3)   # Some useless rows at the beginning of the file
ws_df = pd.read_excel('Kalajoki_Ulkokalla_wind_010121-210924.xlsx') 

print(ep_df.head())
print(50*"-")
print(ws_df.head())


                 Aika  Hinta (snt/kWh)
0 2021-01-01 00:59:11            3.094
1 2021-01-01 01:59:11            3.019
2 2021-01-01 02:59:11            2.974
3 2021-01-01 03:59:11            2.941
4 2021-01-01 04:59:11            2.943
--------------------------------------------------
        Havaintoasema  Vuosi  Kuukausi  Päivä Aika [Paikallinen aika]  \
0  Kalajoki Ulkokalla   2021         1      1                   00:00   
1  Kalajoki Ulkokalla   2021         1      1                   01:00   
2  Kalajoki Ulkokalla   2021         1      1                   02:00   
3  Kalajoki Ulkokalla   2021         1      1                   03:00   
4  Kalajoki Ulkokalla   2021         1      1                   04:00   

  Keskituulen nopeus [m/s] Lämpötilan keskiarvo [°C]  
0                      6.2                         1  
1                      5.6                       0.8  
2                      4.8                       0.7  
3                        5                       0.5  
4

In [71]:
# Format the time column of the dataframes to match one another
from datetime import timedelta
ep_df['Aika'] = ep_df['Aika'] + timedelta(seconds=49)
print(ep_df.head())
print(50*"-")

# Rename wind data time columns to enable application of to_datetime call
ws_df['Aika [Paikallinen aika]'] = ws_df['Aika [Paikallinen aika]'].apply(lambda x: int(x.split(":")[0]))
ws_df.rename(columns={'Vuosi': 'year', 'Kuukausi': 'month', 'Päivä': 'day', 'Aika [Paikallinen aika]': 'hour'}, inplace=True)

ws_df['Aika'] = pd.to_datetime(ws_df[['year', 'month', 'day', 'hour']])
# Remove the now obsolete time-related columns
ws_df.drop(['year','month','day','hour'], axis=1, inplace=True)

print(ws_df.head())

                 Aika  Hinta (snt/kWh)
0 2021-01-01 01:00:00            3.094
1 2021-01-01 02:00:00            3.019
2 2021-01-01 03:00:00            2.974
3 2021-01-01 04:00:00            2.941
4 2021-01-01 05:00:00            2.943
--------------------------------------------------
        Havaintoasema Keskituulen nopeus [m/s] Lämpötilan keskiarvo [°C]  \
0  Kalajoki Ulkokalla                      6.2                         1   
1  Kalajoki Ulkokalla                      5.6                       0.8   
2  Kalajoki Ulkokalla                      4.8                       0.7   
3  Kalajoki Ulkokalla                        5                       0.5   
4  Kalajoki Ulkokalla                      4.6                       0.4   

                 Aika  
0 2021-01-01 00:00:00  
1 2021-01-01 01:00:00  
2 2021-01-01 02:00:00  
3 2021-01-01 03:00:00  
4 2021-01-01 04:00:00  
